# In this notebook we use RAG, I think it's not working as intended

In [1]:
from src.data import load

In [2]:
df_news = load.triplet_news_txt("data/AAPL.txt")
df_news

,title,date_raw,source_code,body
0,APPLE INC : UBS reiterates its Neutral rating,Dec. 09,ZD,APPLE INC : UBS reiterates its Neutral rating
1,Citigroup Adjusts Price Target on Apple to $33...,Dec. 09,MT,Citigroup Adjusts Price Target on Apple to $33...
2,"Apple to Enter AI Revolution in 2026, Wedbush ...",Dec. 08,MT,"Apple to Enter AI Revolution in 2026, Wedbush ..."
3,APPLE INC : JP Morgan remains its Buy rating,Dec. 08,ZD,APPLE INC : JP Morgan remains its Buy rating
4,APPLE INC : Gets a Neutral rating from UBS,Dec. 08,ZD,APPLE INC : Gets a Neutral rating from UBS
...,...,...,...,...
693,Profit momentum of Big Six tech companies to '...,24-04-22,RE,Profit momentum of Big Six tech companies to '...
694,"Analyst recommendations: AMD, Cisco, Alphabet,...",24-04-22,Zonebourse,"Analyst recommendations: AMD, Cisco, Alphabet,..."
695,Morgan Stanley Adjusts Apple's Price Target to...,24-04-22,MT,Morgan Stanley Adjusts Apple's Price Target to...
696,It was better before...,24-04-19,Zonebourse,It was better before...


In [3]:
df_news = df_news.drop(df_news.index[[i for i in range(len(df_news)) if i % 2 == 0 or i % 3 == 0]])
df_news

,title,date_raw,source_code,body
1,Citigroup Adjusts Price Target on Apple to $33...,Dec. 09,MT,Citigroup Adjusts Price Target on Apple to $33...
5,Apple AI Strategy Finally Taking Shape Amid Ke...,Dec. 08,MT,Apple AI Strategy Finally Taking Shape Amid Ke...
7,Wedbush Ups PT on Apple to $350 From $320 on '...,Dec. 08,MT,Wedbush Ups PT on Apple to $350 From $320 on '...
11,MoffettNathanson Adjusts Price Target on Apple...,Dec. 03,MT,MoffettNathanson Adjusts Price Target on Apple...
13,Loop Capital Adjusts Price Target on Apple to ...,Dec. 02,MT,Loop Capital Adjusts Price Target on Apple to ...
...,...,...,...,...
685,APPLE INC : UBS sticks Neutral,24-04-29,ZD,APPLE INC : UBS sticks Neutral
689,"Alphabet, Microsoft shares jump on proof of ne...",24-04-26,RE,"Alphabet, Microsoft shares jump on proof of ne..."
691,APPLE INC : UBS reiterates its Neutral rating,24-04-23,ZD,APPLE INC : UBS reiterates its Neutral rating
695,Morgan Stanley Adjusts Apple's Price Target to...,24-04-22,MT,Morgan Stanley Adjusts Apple's Price Target to...


In [4]:
import os
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "gemma3:1b")
EMBED_MODEL = os.getenv("OLLAMA_EMBED_MODEL", "mxbai-embed-large")

In [7]:
from src.rag import trying

In [8]:
news_index = trying.build_news_rag_index(
    df_news,
    text_col="body",        # we embedded the headline text in 'body'
    embed_model=EMBED_MODEL # e.g. "mxbai-embed-large"
)


[build_news_rag_index] Embedding 233 headlines with mxbai-embed-large...


In [ ]:
headline = df_news.loc[1, "body"]

res = trying.classify_headline_sentiment_ollama_rag(
    headline,
    index=news_index,
    model=OLLAMA_MODEL,  # e.g. "qwen3:4b"
    k=3                  # number of similar headlines as context
)

res


{'sentiment': 'bullish',
 'score': 0.85,
 'confidence': 0.9,
 'rationale': "The headline explicitly states an increase in Apple's price target, suggesting a positive outlook for its future performance. It's a significant upward adjustment.",
 'raw': '```json\n{\n  "sentiment": "bullish",\n  "score": 0.85,\n  "confidence": 0.9,\n  "rationale": "The headline explicitly states an increase in Apple\'s price target, suggesting a positive outlook for its future performance. It\'s a significant upward adjustment."\n}\n```'}

In [ ]:
df_scored_rag = trying.core_news_df_with_ollama_rag(
    df_news,
    index=news_index,
    text_col="body",      # input text
    model=OLLAMA_MODEL,   # "qwen3:4b"
    limit=None,           # or an int like 3 for quick tests
    k=3                   # top-3 retrieved headlines as context
)

df_scored_rag[
    ["date_raw", "source_code", "title",
     "sentiment", "sentiment_score", "sentiment_confidence"]
]

[score_news_df_with_ollama_rag] 1/233: Citigroup Adjusts Price Target on Apple to $330 From $315, Maintains Buy Rating...
[score_news_df_with_ollama_rag] 2/233: Apple AI Strategy Finally Taking Shape Amid Key Hire, Expected Exclusive Partnership With ...
[score_news_df_with_ollama_rag] 3/233: Wedbush Ups PT on Apple to $350 From $320 on 'Solid' iPhone 17 Sales, Says AI Efforts to A...
[score_news_df_with_ollama_rag] 4/233: MoffettNathanson Adjusts Price Target on Apple to $234 From $225, Maintains Neutral Rating...
[score_news_df_with_ollama_rag] 5/233: Loop Capital Adjusts Price Target on Apple to $325 From $315, Maintains Buy Rating...
[score_news_df_with_ollama_rag] 6/233: APPLE INC : JP Morgan remains its Buy rating...
[score_news_df_with_ollama_rag] 7/233: Nvidia Sell-Off a 'DeepSeek Moment,' AI Revolution Just Beginning, Wedbush Says...
[score_news_df_with_ollama_rag] 8/233: APPLE INC : Jefferies is Neutral...
[score_news_df_with_ollama_rag] 9/233: Sector Update: Tech Stocks Mixe

,date_raw,source_code,title,sentiment,sentiment_score,sentiment_confidence
1,Dec. 09,MT,Citigroup Adjusts Price Target on Apple to $33...,bullish,0.85,0.90
5,Dec. 08,MT,Apple AI Strategy Finally Taking Shape Amid Ke...,bullish,0.85,0.95
7,Dec. 08,MT,Wedbush Ups PT on Apple to $350 From $320 on '...,bullish,0.85,0.95
11,Dec. 03,MT,MoffettNathanson Adjusts Price Target on Apple...,bullish,0.75,0.90
13,Dec. 02,MT,Loop Capital Adjusts Price Target on Apple to ...,bullish,0.85,0.90
...,...,...,...,...,...,...
685,24-04-29,ZD,APPLE INC : UBS sticks Neutral,bullish,0.85,0.90
689,24-04-26,RE,"Alphabet, Microsoft shares jump on proof of ne...",bullish,0.85,0.90
691,24-04-23,ZD,APPLE INC : UBS reiterates its Neutral rating,neutral,0.10,0.90
695,24-04-22,MT,Morgan Stanley Adjusts Apple's Price Target to...,bullish,0.85,0.90


In [ ]:
df_scored_rag.to_csv("data/AAPL_sentiment_rag.csv", index=False)